In [23]:
#-*- coding:utf-8 -*-
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import torch.utils.data as data

train = pd.read_csv("data.csv",parse_dates=["Date"])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
wind1 = pd.read_csv("./dataset/wind1.csv")
wind2 = pd.read_csv("./dataset/wind2.csv")
wind3 = pd.read_csv("./dataset/wind3.csv")
wind4 = pd.read_csv("./dataset/wind4.csv")

In [24]:
from torch.optim.lr_scheduler import ExponentialLR

# Hyper-Parameters
sequence_length = 36
input_size = 2
hidden_size = 64
num_layers = 3
output_size = 4
batch_size = 64
num_epochs = 32
learning_rate = 1e-5

In [25]:
class mydata(data.Dataset):
    def __init__(self, input_dat, y,seq_length):
        self.input_dat = torch.tensor(input_dat).float()
        self.y = torch.tensor(y).float()
        self.seq_length = seq_length

    def __getitem__(self, idx):
        if idx >= self.seq_length - 1:
            i_start = idx - self.seq_length + 1
            x = self.input_dat[i_start:(idx-3),:]
            y = self.y[(idx-3):(idx+1)]
        elif idx>=3:
            padding = self.input_dat[0].repeat(self.seq_length-idx-1,1)
            x = self.input_dat[0:(idx-3),:]
            x = torch.cat((padding,x),0)
            y = self.y[(idx-3):(idx+1)]
        else:
            x = self.input_dat[0].repeat(self.seq_length-4,1)
            padding_y = self.y[0].repeat(3-idx)
            y = self.y[0:(idx+1)]
            y = torch.cat((padding_y,y),0)
        return x, y
    
    def __len__(self):
        return self.input_dat.shape[0]

Using two strategies,

1. Test using MA data and Test relative error using MA data
2. Test using original data and test re using original data

Later, I'll figure out how to test using original data

Split the 4th dataset to test set and the first 3 dataset as training set.

In [26]:
# Training Sets
train1 = np.array(wind1[["GridPowerMA","WindSpdMA"]][19:])
train_data1 = mydata(train1, train1[:,1], sequence_length)
train2 = np.array(wind2[["GridPowerMA","WindSpdMA"]][19:])
train_data2 = mydata(train2, train2[:,1], sequence_length)
train3 = np.array(wind3[["GridPowerMA","WindSpdMA"]][19:])
train_data3 = mydata(train3, train3[:,1], sequence_length)

# Test Sets
test = np.array(wind4[["GridPowerMA","WindSpdMA"]][19:])
test_target = np.array(wind4["GridPower"][19:])
test_data = mydata(test, test_target, sequence_length)
test_data1 = mydata(test,test[:,1],sequence_length)

# Data Loader
train_dat1 = data.DataLoader(dataset=train_data1, batch_size=batch_size, shuffle=True)
train_dat2 = data.DataLoader(dataset=train_data2, batch_size=batch_size, shuffle=True)
train_dat3 = data.DataLoader(dataset=train_data3, batch_size=batch_size, shuffle=True)
test_dat = data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)
test_dat1 = data.DataLoader(dataset=test_data1, batch_size=batch_size, shuffle=True)

In [27]:
##Build RNN(LSTM)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size,output_size)

    def forward(self, x):
        # set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

lstm_ma = RNN(input_size, hidden_size, num_layers, output_size).to(device)
lstm_o = RNN(input_size,hidden_size, num_layers, output_size).to(device)

# Loss and optimizer
criterion = nn.MSELoss()
op_ma = torch.optim.Adam(lstm_ma.parameters(), lr=learning_rate)
op_o = torch.optim.Adam(lstm_ma.parameters(), lr=learning_rate)
scheduler_ma = ExponentialLR(op_ma,gamma=0.9)
scheduler_o = ExponentialLR(op_o,gamma=0.9)

In [28]:
# Training
for epoch in range(num_epochs):
    for i, (value,labels) in enumerate(train_dat1):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_ma.zero_grad()
        loss.backward()
        op_ma.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat2):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_ma.zero_grad()
        loss.backward()
        op_ma.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat3):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_ma.zero_grad()
        loss.backward()
        op_ma.step()

        if i % 500 == 0:
            print(loss.item())
    
    scheduler_ma.step()

0.23247407376766205
0.1668686419725418
0.10178763419389725
0.19379261136054993
0.168056458234787
0.09829328954219818
0.2280089110136032
0.12944704294204712
0.09611589461565018
0.20459991693496704
0.16024570167064667
0.09389175474643707
0.20289510488510132
0.13519279658794403
0.09103573858737946
0.195567786693573
0.13120624423027039
0.0902961790561676
0.18834760785102844
0.13051503896713257
0.08765137195587158
0.22784999012947083
0.1272963434457779
0.08554662764072418
0.1880972534418106
0.1319674253463745
0.0844850167632103
0.17595909535884857
0.1327458620071411
0.08339233696460724
0.17157359421253204
0.12328625470399857
0.07940793037414551
0.16686026751995087
0.12831765413284302
0.07961888611316681
0.21523205935955048
0.1271696835756302
0.07895112782716751
0.16976554691791534
0.1163848340511322
0.07773329317569733
0.1579831838607788
0.11638255417346954
0.07796187698841095
0.19742295145988464
0.12467637658119202
0.07739713042974472
0.16481447219848633
0.11793337762355804
0.0762011036276

In [29]:
# Evaluation
lstm_ma.eval()
mse = 0
i = 0
re = np.array([])
with torch.no_grad():
    for values, labels in test_dat:
        values = values.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        outputs = lstm_ma(values)
        mse += torch.sum((outputs-labels)**2)
        i += len(labels.flatten())
        re_tmp = np.array(np.abs(labels.flatten()-outputs.flatten()))
        re = np.concatenate((re,re_tmp),axis=0)
print(mse/i)
mean_rerror = np.mean(re)
max_rerror = np.max(re)
median_rerror = np.median(re)
print(mean_rerror)
print(max_rerror)
print(median_rerror)
idx = re <= 0.15
idx = idx + 0
print(sum(idx)/len(idx))

tensor(0.1590)
0.29496159076306705
1.038631796836853
0.22426925599575043
0.3798245614035088


In [30]:
# Evaluation using ma data
lstm_ma.eval()
mse = 0
i = 0
re = np.array([])
with torch.no_grad():
    for values, labels in test_dat1:
        values = values.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        outputs = lstm_ma(values)
        mse += torch.sum((outputs-labels)**2)
        i += len(labels.flatten())
        re_tmp = np.array(np.abs(labels.flatten()-outputs.flatten()))
        re = np.concatenate((re,re_tmp),axis=0)
print(mse/i)
mean_rerror = np.mean(re)
max_rerror = np.max(re)
median_rerror = np.median(re)
print(mean_rerror)
print(max_rerror)
print(median_rerror)
idx = re <= 0.15
idx = idx + 0
print(sum(idx)/len(idx))

tensor(0.1182)
0.32495330091761915
0.6254315376281738
0.3184541016817093
0.043859649122807015


If just input the original data, let's see what will happen.

In [31]:
# Training Sets
train1 = np.array(wind1[["GridPower","WindSpd"]][19:])
train_data1 = mydata(train1, train1[:,1], sequence_length)
train2 = np.array(wind2[["GridPower","WindSpdMA"]][19:])
train_data2 = mydata(train2, train2[:,1], sequence_length)
train3 = np.array(wind3[["GridPower","WindSpd"]][19:])
train_data3 = mydata(train3, train3[:,1], sequence_length)

# Data Loader
train_dat1 = data.DataLoader(dataset=train_data1, batch_size=batch_size, shuffle=True)
train_dat2 = data.DataLoader(dataset=train_data2, batch_size=batch_size, shuffle=True)
train_dat3 = data.DataLoader(dataset=train_data3, batch_size=batch_size, shuffle=True)
test_dat = data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)
test_dat1 = data.DataLoader(dataset=test_data1, batch_size=batch_size, shuffle=True)

In [32]:
# Training
for epoch in range(num_epochs):
    for i, (value,labels) in enumerate(train_dat1):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_o(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_o.zero_grad()
        loss.backward()
        op_o.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat2):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_o(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_o.zero_grad()
        loss.backward()
        op_o.step()

        if i % 500 == 0:
            print(loss.item())
    
    for i, (value,labels) in enumerate(train_dat3):
        value = value.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs = lstm_ma(value)
        loss = criterion(outputs.squeeze(),labels.squeeze())

        # optimization step
        op_o.zero_grad()
        loss.backward()
        op_o.step()

        if i % 500 == 0:
            print(loss.item())
    
    scheduler_o.step()

0.15396609902381897
0.12202358990907669
0.07212130725383759
0.1529170274734497
0.12267377972602844
0.07155870646238327
0.18338389694690704
0.13738606870174408
0.0696970745921135
0.18831761181354523
0.15084999799728394
0.07218291610479355
0.17475491762161255
0.12089580297470093
0.06876976788043976
0.18361428380012512
0.140211284160614
0.06818101555109024
0.1825834959745407
0.13150930404663086
0.06813901662826538
0.1979655623435974
0.12256109714508057
0.06794748455286026
0.16829614341259003
0.13042856752872467
0.06849399209022522
0.17424876987934113
0.1392582654953003
0.06697522848844528
0.17762917280197144
0.1249767318367958
0.06696401536464691
0.17472949624061584
0.11461129039525986
0.06643486768007278
0.19976477324962616
0.1246166080236435
0.06656629592180252
0.18946950137615204
0.13378331065177917
0.06606850028038025
0.18713048100471497
0.11377917975187302
0.0667945146560669
0.17057552933692932
0.12270572781562805
0.06652942299842834
0.17585238814353943
0.12238942831754684
0.06427181

In [33]:
# Evaluation
lstm_ma.eval()
mse = 0
i = 0
re = np.array([])
with torch.no_grad():
    for values, labels in test_dat:
        values = values.reshape(-1,sequence_length-4,input_size).to(device)
        labels = labels.to(device)

        outputs = lstm_ma(values)
        mse += torch.sum((outputs-labels)**2)
        i += len(labels.flatten())
        re_tmp = np.array(np.abs(labels.flatten()-outputs.flatten()))
        re = np.concatenate((re,re_tmp),axis=0)
print(mse/i)
mean_rerror = np.mean(re)
max_rerror = np.max(re)
median_rerror = np.median(re)
print(mean_rerror)
print(max_rerror)
print(median_rerror)
idx = re <= 0.15
idx = idx + 0
print(sum(idx)/len(idx))

tensor(0.1494)
0.28150312708805136
1.0250438451766968
0.2079901546239853
0.4070175438596491
